# The aim of this notebook is to implement the work presented in the article [here](https://towardsdatascience.com/how-to-train-multiple-machine-learning-models-and-run-other-data-tasks-in-parallel-by-combining-2fa9670dd579)

## Reading in packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
import pathlib
import shutil
import tempfile
import vaex
from IPython import display
from IPython.display import clear_output
import pyodbc

# Tensorflow related
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow.compat.v2.feature_column as fc

#!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print(tf.__version__)
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)
print(logdir)

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#sys.path.append('...../')

from utils.data_extraction import load_data_final,load_data_and_save
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final,\
                                    generateFeatures_multi_v2

from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing,\
                                                    extract_labels_multi_final,\
                                                    align_features_and_labels_multi_final,\
                                                    align_features_and_labels_multi_v5

from utils.models import make_input_fn
from utils.models import performanceTesting,scoreFunction
from utils.plotting import plot_confusion_matrix

2.2.0
C:\Users\PC\AppData\Local\Temp\tmps3_ek40c\tensorboard_logs


In [79]:
def align_features_and_labels_multi_v7(price_candles,
                                        all_features,
                                        prediction_horizon,
                                        n_feature_lags,
                                        n_classes,
                                        safe_burn_in = False,
                                        data_sample = 'full',
                                        splitType='global',
                                        noise = False,
                                        ticker_dummies = False):

    all_burned_in_features = pd.DataFrame()
    all_burned_in_indices = pd.DataFrame()
    all_labels = pd.DataFrame()
    
    dailyIndices = pd.DataFrame({'days':price_candles.index.get_level_values(0),
                                      'timestemps':price_candles.index.get_level_values(1),
                                      'ticker':price_candles.Ticker})

    if splitType.lower() == 'global':
        # Making the splits for the labels based on all tickers
        # returns = ((price_candles['close'].values[1:] / price_candles['close'].values[:-1]) -1) * 100
#         returns = np.concatenate([((price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
#                          price_candles[price_candles.Ticker==ticker]['close'].values[:-1])-1) for ticker\
#                           in price_candles.Ticker.unique()])

        returns = []
        tickers = []
        
        for ticker in price_candles.Ticker.unique():

            ticker_returns = (price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
                                 price_candles[price_candles.Ticker==ticker]['close'].values[:-1]) - 1
            ticker_names = [ticker for i in range(len(ticker_returns))]

            returns.append(ticker_returns)
            tickers.append(ticker_names)

        # concatenate returns and add noise
        returns = np.concatenate(returns)
        if noise:
            returns[returns==0] = np.random.normal(0,1,sum(returns==0))/1000000

        tickers = np.concatenate(tickers)

        _, splits = pd.qcut(returns, q=n_classes, labels=False, retbins=True)
        #print(splits)

        returns = pd.DataFrame({'returns': returns, 'Ticker': tickers})

    keepCheck = []

    for ticker_iter, ticker_name in enumerate(all_features.ticker.unique()):
        ticker_features = all_features[all_features.ticker==ticker_name].copy(deep=True)
        ticker_indices = dailyIndices[dailyIndices.ticker==ticker_name].copy(deep=True)
        # removing the "ticker" variable from ticker_features as np.isnan() does not like non-numericals
        #ticker_features = ticker_features.iloc[:, ticker_features.columns != 'ticker']
        ticker_features.drop('ticker', axis=1, inplace=True)
        # extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
        #ticker_prices = price_candles[price_candles.Ticker==ticker_name]['close'].values # candles.iloc[:, :4].values
        ticker_returns = returns[returns.Ticker==ticker_name]['returns'].values

        if not safe_burn_in:
            assert data_sample == 'full'
            # we assume data_sample is full and that we can continue features from yesterday's values.
            # that we have a single burn-in at the beginning and that's it

            # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
            burned_in_idx = np.where((np.sum(np.isnan(ticker_features.values), axis=1) == 0) == True)[0][0]
            keepCheck.append(burned_in_idx)
            # calculate end-point cut-off to match with labels
            end_point_cut = max(prediction_horizon, n_feature_lags + 1)

            # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
            burned_in_features = ticker_features.iloc[burned_in_idx : -end_point_cut, :] #.reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
            burned_in_indices = ticker_indices.iloc[burned_in_idx : -end_point_cut, :]
            # slice away the burned-in indices from labels
            labels = extract_labels_multi_final(data = ticker_returns[(burned_in_idx+n_feature_lags):],
                                                classes = n_classes,
                                                group_style = 'equal',
                                                splits = splits)
            # labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + n_feature_lags : , :],
            #                                             classes = n_classes, group_style = 'equal')

            # check if there are remaining NaNs are burn-in (means error)
            remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
            if remaining_nans > 0:
                raise ValueError('Had NaN in burned_in_features after burn-in')

        # Adding the ticker
        burned_in_features.loc[:,'ticker'] = ticker_name

        # Adding the burned in data
        all_burned_in_features = pd.concat([all_burned_in_features, burned_in_features.reset_index(drop=True)])
        all_burned_in_indices = pd.concat([all_burned_in_indices, burned_in_indices.reset_index(drop=True)])
        all_labels = pd.concat([all_labels, pd.Series(labels)])
        print(ticker_name + " done")

    # Returning the ticker as dummies
    if ticker_dummies:

        tickers = all_burned_in_features.pop('ticker')
        all_burned_in_features = pd.concat([all_burned_in_features, pd.get_dummies(tickers, prefix='d_ticker', drop_first=False)], axis=1)
#     print('Are all burned_in_idx the same?', all(keepCheck==keepCheck[0]))
#     print(dailyIndicies.head(50))
    return all_burned_in_features.reset_index(drop=True),\
            all_labels.reset_index(drop=True),\
            all_burned_in_indices.reset_index(drop=True)

## Extracting data

In [65]:
# Do we extract new data or read in?
readIn = True
# run load_data()
if readIn:
    
    # Listing the data files 
    path = '../../../Google Drev/Thesis/Data/TAQ/AggregatedTAQ'
#     path = 'F:/AggregatedTAQ/round3'
    datafiles = os.listdir(path)
    content = np.concatenate([['\n\n'],[str(j)+': '+i+'\n' for j,i in enumerate(datafiles) if 'csv' in i],['\n\n']])
    
    # Asking for user input
    file = input('Which one do you want to load? %s'%''.join(content))
    data = pd.read_csv(path + '/' + datafiles[int(file)],
                       header = None,
                       names=['open','high','low','close',
                              'spread_open','spread_high','spread_low','spread_close',
                              'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                              'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                              'Ticker'])
    # Lower casing all column names
#     data.columns = data.columns.str.lower()
else:
    
    # print(os.listdir())
    try:
        path = 'a:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
    except:
        path = 't:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
        
    # Sample type
    data_sample = 'full' # or 'stable'
    # allFiles = os.listdir(path)
    # print(len(allFiles), allFiles[:5], allFiles[-5:])
    # print(allFiles[-10:])

    #dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
    dates = np.array(['20200501']).astype(int)#,'20200402','20200403','20200406','20200407'

    # Provide a list of tickers of interest
    
    tickers = sorted(['TSLA','FB'])#'MSFT'
    
    # Do we need data on trades, quotes or both?
    dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'
    
    if dataNeeded == 'trades':
        tradeData = load_data_final(dates, tickers, dataNeeded, path, verbose)
    elif dataNeeded == 'quotes':
        quoteData = load_data_final(dates,
                                    tickers,
                                    dataNeeded,
                                    path,
                                    verbose,
                                    extract_candles = False,
                                    aggHorizon = 1,
                                    extra_features_from_quotes = None,
                                    data_sample = data_sample)
    elif dataNeeded == 'both':
        tradeData, quoteData = load_data_final(dates, tickers, dataNeeded, path, verbose)

# Reading in sector information
stockInfo = pd.read_csv('../utils/stockInfo_v1.csv',header=[0,1])
stockInfo.columns = ['ticker','sector','exchange','marketCap']

# Creating a table with stock information based on the tickers available in the data.
uniqueTickers = data.Ticker.unique()
stockTable = stockInfo[stockInfo.ticker.isin(uniqueTickers)]
stockTable.head(10)

Which one do you want to load? 

0: aggregateTAQ_May2020_10sec (1).csv
1: aggregateTAQ_May2020_30sec (1).csv
2: aggregateTAQ_May2020_60sec.csv


2


,ticker,sector,exchange,marketCap
12,AAPL,Technology,NMS,1.578173e+12
20,ABBV,Healthcare,NYQ,1.742612e+11
34,ABT,Healthcare,NYQ,1.631410e+11
126,AEP,Utilities,NYQ,4.089551e+10
379,AMT,Real Estate,NYQ,1.171259e+11
428,APD,Basic Materials,NYQ,5.464395e+10
697,BA,Industrials,NYQ,1.020356e+11
699,BABA,Consumer Cyclical,NYQ,5.936536e+11
700,BAC,Financial Services,NYQ,2.020550e+11
870,BHP,Basic Materials,NYQ,1.258194e+11


### Dropping ETFS

In [3]:
etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF']

# Extracting the sector ETFs to a separate variable
sectorETFS = data[data.Ticker.isin(etfs)]

# Removing the ETFs
data = data[~data.Ticker.isin(etfs)]

## Generating Features

In [80]:
########### Generate Features ################

n_feature_lags = 1

features = generateFeatures_multi_v2(data = data, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'spread',
                                                    'bidsize',
                                                    'ofrsize',
#                                                     'stok',
#                                                     'stod',
#                                                     'sstod',
#                                                     'wilr',
#                                                     'roc',
#                                                     'rsi',
#                                                     'atr',
#                                                     'cci',
#                                                     'dpo',
#                                                     'sma',
#                                                     'ema',
#                                                     'macd',
#                                                       'macd_diff',
#                                                       'macd_signal',
#                                                     'dis5',
#                                                     'dis10',
                                                      'sector'
                                                   ], 
                                   feature_lags = n_feature_lags
                                     ,stockTable=stockTable)

########### Generate Labels ################

n_classes = 2
# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
# X, y = align_features_and_labels_multi_final(price_candles = price_candles, 
#                                              all_features = features,
#                                              prediction_horizon = 1, 
#                                              n_feature_lags = n_feature_lags, 
#                                              n_classes = n_classes, # 5,
#                                              safe_burn_in = False, 
#                                              data_sample = 'full',
#                                              splitType='global',
#                                              noise=False,ticker_dummies=False)

X_temp, y_temp,indices = align_features_and_labels_multi_v7(price_candles = price_candles, 
                                                             all_features = features,
                                                             prediction_horizon = 1, 
                                                             n_feature_lags = n_feature_lags, 
                                                             n_classes = n_classes, # 5,
                                                             safe_burn_in = False, 
                                                             data_sample = 'full',
                                                             splitType='global',
                                                             noise=False,ticker_dummies=False)

C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\PC\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


AAPL done
ABBV done
ABT done
AEP done
AMT done
APD done
BA done
BABA done
BAC done
BHP done
BP done
CCI done
CHL done
COST done
CSGP done
D done
DIS done
ECL done
ENB done
EXC done
FB done
FMX done
GOOG done
IDU done
INTC done
IYC done
IYE done
IYG done
IYH done
IYJ done
IYK done
IYM done
IYR done
IYW done
IYZ done
JNJ done
KO done
LFC done
LIN done
LMT done
MA done
MCD done
MSFT done
NKE done
NVDA done
NVS done
Number of NaNs in label: 1. 1 is expected
Returns that lead to NaNs in label: [0.0907158]
PBR done
PEP done
PFE done
PLD done
PSA done
PTR done
PYPL done
RTX done
SHW done
SNP done
SO done
SRE done
T done
TM done
TSLA done
TSM done
UNP done
UPS done
V done
WMT done


In [81]:
indices

,days,timestemps,ticker
0,20200501,0,AAPL
1,20200501,1,AAPL
2,20200501,2,AAPL
3,20200501,3,AAPL
4,20200501,4,AAPL
...,...,...,...
514663,20200529,383,WMT
514664,20200529,384,WMT
514665,20200529,385,WMT
514666,20200529,386,WMT


In [82]:
X_temp

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
1,0.455,1.125,-0.300,288.580,0.07,0.49,0.01,0.30,1.0,50.0,...,0,0,0,0,0,0,0,1,0,AAPL
2,-0.610,0.220,-0.815,289.095,0.49,0.49,0.01,0.17,1.0,25.0,...,0,0,0,0,0,0,0,1,0,AAPL
3,-1.220,0.115,-1.380,290.320,0.16,0.33,0.01,0.10,13.0,71.0,...,0,0,0,0,0,0,0,1,0,AAPL
4,0.235,0.355,-0.565,290.085,0.10,0.42,0.01,0.05,2.0,86.0,...,0,0,0,0,0,0,0,1,0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514663,-0.110,0.010,-0.175,123.950,0.02,0.07,0.01,0.02,3.0,6.0,...,0,1,0,0,0,0,0,0,0,WMT
514664,-0.150,0.010,-0.190,124.100,0.02,0.07,0.01,0.04,1.0,11.0,...,0,1,0,0,0,0,0,0,0,WMT
514665,0.090,0.090,-0.075,123.995,0.01,0.06,0.01,0.01,1.0,8.0,...,0,1,0,0,0,0,0,0,0,WMT
514666,-0.340,0.020,-0.350,124.335,0.01,0.07,0.01,0.05,4.0,16.0,...,0,1,0,0,0,0,0,0,0,WMT


In [83]:
X

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
1,0.455,1.125,-0.300,288.580,0.07,0.49,0.01,0.30,1.0,50.0,...,0,0,0,0,0,0,0,1,0,AAPL
2,-0.610,0.220,-0.815,289.095,0.49,0.49,0.01,0.17,1.0,25.0,...,0,0,0,0,0,0,0,1,0,AAPL
3,-1.220,0.115,-1.380,290.320,0.16,0.33,0.01,0.10,13.0,71.0,...,0,0,0,0,0,0,0,1,0,AAPL
4,0.235,0.355,-0.565,290.085,0.10,0.42,0.01,0.05,2.0,86.0,...,0,0,0,0,0,0,0,1,0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514663,-0.110,0.010,-0.175,123.950,0.02,0.07,0.01,0.02,3.0,6.0,...,0,1,0,0,0,0,0,0,0,WMT
514664,-0.150,0.010,-0.190,124.100,0.02,0.07,0.01,0.04,1.0,11.0,...,0,1,0,0,0,0,0,0,0,WMT
514665,0.090,0.090,-0.075,123.995,0.01,0.06,0.01,0.01,1.0,8.0,...,0,1,0,0,0,0,0,0,0,WMT
514666,-0.340,0.020,-0.350,124.335,0.01,0.07,0.01,0.05,4.0,16.0,...,0,1,0,0,0,0,0,0,0,WMT


## Can we replicate the below now?

In [84]:
data

open     high      low    close  spread_open  spread_high  \
20200501 0    286.250  289.260  285.870  289.260         0.50         0.50   
         1    289.260  289.350  288.365  289.020         0.24         0.45   
         2    289.035  289.705  288.280  288.580         0.07         0.49   
         3    288.485  289.315  288.280  289.095         0.49         0.49   
         4    289.100  290.435  288.940  290.320         0.16         0.33   
...               ...      ...      ...      ...          ...          ...   
20200529 385  123.950  124.110  123.910  124.100         0.02         0.07   
         386  124.085  124.085  123.920  123.995         0.01         0.06   
         387  123.995  124.355  123.985  124.335         0.01         0.07   
         388  124.335  124.355  124.060  124.075         0.05         0.12   
         389  124.075  124.225  122.810  123.855         0.01         2.43   

              spread_low  spread_close  bidsize_open  bidsize_high  \
20200501 0          0.01          0.24           6.0          95.0   
         1          0.01          0.10           9.0          20.0   
         2          0.01          0.30           1.0          50.0   
         3          0.01          0.17           1.0          25.0   
         4          0.01          0.10          13.0          71.0   
...                  ...           ...           ...           ...   
20200529 385        0.01          0.04           1.0          11.0   
         386        0.01          0.01           1.0           8.0   
         387        0.01          0.05           4.0          16.0   
         388        0.01          0.01           3.0           6.0   
         389        0.01          0.21           1.0          20.0   

              bidsize_low  bidsize_close  ofrsize_open  ofrsize_high  \
20200501 0            1.0           10.0           1.0          85.0   
         1            1.0            1.0           4.0          56.0   
         2            1.0            1.0           1.0          13.0   
         3            1.0           16.0           1.0           8.0   
         4            1.0            1.0           1.0         236.0   
...                   ...            ...           ...           ...   
20200529 385          1.0            1.0           5.0           9.0   
         386          1.0            3.0           1.0           9.0   
         387          1.0            2.0           2.0          10.0   
         388          1.0            2.0           2.0          10.0   
         389          1.0            2.0           4.0          12.0   

              ofrsize_low  ofrsize_close Ticker  
20200501 0            1.0            4.0   AAPL  
         1            1.0            1.0   AAPL  
         2            1.0            1.0   AAPL  
         3            1.0            1.0   AAPL  
         4            1.0            1.0   AAPL  
...                   ...            ...    ...  
20200529 385          1.0            1.0    WMT  
         386          1.0            2.0    WMT  
         387          1.0            2.0    WMT  
         388          1.0            4.0    WMT  
         389          1.0            1.0    WMT  

[514800 rows x 17 columns]

In [90]:
data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1'])

open     high      low    close  spread_open  \
level_0  level_1                                                    
20200501 0        286.250  289.260  285.870  289.260         0.50   
         0         81.000   82.110   80.580   81.010         2.00   
         0         91.025   91.435   90.490   91.220         1.89   
         0         83.455   83.980   82.250   82.470         3.09   
         0        234.350  234.575  232.215  232.950         3.30   
...                   ...      ...      ...      ...          ...   
20200529 389       50.455   50.495   50.325   50.325         0.01   
         389      169.960  170.030  169.610  169.675         0.04   
         389      100.095  100.110   99.670   99.670         0.05   
         389      195.855  195.920  194.195  194.195         0.03   
         389      124.075  124.225  122.810  123.855         0.01   

                  spread_high  spread_low  spread_close  bidsize_open  \
level_0  level_1                                                        
20200501 0               0.50        0.01          0.24           6.0   
         0               2.06        0.01          0.24          12.0   
         0               1.89        0.03          0.24           1.0   
         0               4.14        0.10          0.14           1.0   
         0               3.87        0.27          2.90           1.0   
...                       ...         ...           ...           ...   
20200529 389             0.05        0.01          0.05           5.0   
         389             0.36        0.01          0.23          10.0   
         389             0.13        0.01          0.12           1.0   
         389             2.39        0.01          2.39           2.0   
         389             2.43        0.01          0.21           1.0   

                  bidsize_high  bidsize_low  bidsize_close  ofrsize_open  \
level_0  level_1                                                           
20200501 0                95.0          1.0           10.0           1.0   
         0                12.0          1.0            1.0           1.0   
         0                 5.0          1.0            1.0           1.0   
         0                 7.0          1.0            1.0           7.0   
         0                 3.0          1.0            1.0           2.0   
...                        ...          ...            ...           ...   
20200529 389              38.0          1.0            2.0           7.0   
         389              11.0          1.0            1.0           1.0   
         389              11.0          1.0            7.0           1.0   
         389               7.0          1.0            1.0           1.0   
         389              20.0          1.0            2.0           4.0   

                  ofrsize_high  ofrsize_low  ofrsize_close Ticker  
level_0  level_1                                                   
20200501 0                85.0          1.0            4.0   AAPL  
         0                 9.0          1.0            1.0   ABBV  
         0                 5.0          1.0            1.0    ABT  
         0                 8.0          1.0            1.0    AEP  
         0                 3.0          1.0            1.0    AMT  
...                        ...          ...            ...    ...  
20200529 389              24.0          1.0            9.0    TSM  
         389              10.0          1.0            2.0    UNP  
         389              11.0          1.0            1.0    UPS  
         389               9.0          1.0            2.0      V  
         389              12.0          1.0            1.0    WMT  

[514800 rows x 17 columns]

In [94]:
# Let's see number 1
data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1'])[['close']][(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(1)==1)&(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(0)==20200501)]

close
level_0  level_1         
20200501 1        289.020
         1         81.760
         1         91.220
         1         82.370
         1        233.540
...                   ...
         1         52.440
         1        156.095
         1         93.445
         1        175.260
         1        121.080

[66 rows x 1 columns]

In [95]:
X.loc[indices.sort_values(['days','timestemps','ticker']).index,:].head(66)

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
7798,-0.750,0.310,-0.750,81.760,0.24,1.06,0.07,0.80,14.0,20.0,...,0,0,0,0,1,0,0,0,0,ABBV
15596,-0.030,0.160,-0.320,91.220,0.18,0.58,0.05,0.12,1.0,3.0,...,0,0,0,0,1,0,0,0,0,ABT
23394,0.105,0.105,-0.100,82.370,0.13,0.54,0.10,0.26,1.0,2.0,...,0,0,0,0,0,0,0,0,1,AEP
31192,-0.335,0.685,-0.380,233.540,2.39,3.99,1.42,2.10,1.0,1.0,...,0,0,0,0,0,0,1,0,0,AMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475678,-0.110,0.070,-0.115,52.440,0.06,0.17,0.01,0.08,1.0,14.0,...,0,0,0,0,0,0,0,1,0,TSM
483476,0.670,0.720,-0.110,156.095,0.47,0.71,0.22,0.25,1.0,2.0,...,0,0,0,0,0,1,0,0,0,UNP
491274,-0.430,0.000,-0.710,93.445,1.03,1.10,0.05,0.65,1.0,6.0,...,0,0,0,0,0,1,0,0,0,UPS
499072,-0.365,0.170,-0.365,175.260,0.15,0.23,0.01,0.06,1.0,4.0,...,0,0,0,1,0,0,0,0,0,V


In [97]:
X.loc[indices.sort_values(['days','timestemps','ticker']).index,:].head(66)['close_lag0'].values

array([ 289.02 ,   81.76 ,   91.22 ,   82.37 ,  233.54 ,  222.465,
        142.89 ,  196.92 ,   23.345,   38.985,   23.145,  156.54 ,
         39.255,  301.76 ,  638.615,   76.18 ,  105.765,  189.35 ,
         30.085,   36.375,  202.68 ,   63.23 , 1332.65 ,  140.335,
         58.82 ,  200.515,   19.71 ,  112.33 ,  210.23 ,  135.415,
        117.915,   79.68 ,   73.905,  230.02 ,   27.045,  148.905,
         45.47 ,   10.29 ,  183.25 ,  387.765,  269.125,  184.39 ,
        176.515,   85.27 ,  286.32 ,   84.21 ,    6.735,  131.27 ,
         37.785,   87.1  ,  179.82 ,   34.92 ,  120.815,   63.29 ,
        527.2  ,   48.475,   55.845,  121.405,   30.685,  121.725,
        757.875,   52.44 ,  156.095,   93.445,  175.26 ,  121.08 ])

In [99]:
data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1'])[['close']][(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(1)==1)&(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(0)==20200501)].values.T

array([[ 289.02 ,   81.76 ,   91.22 ,   82.37 ,  233.54 ,  222.465,
         142.89 ,  196.92 ,   23.345,   38.985,   23.145,  156.54 ,
          39.255,  301.76 ,  638.615,   76.18 ,  105.765,  189.35 ,
          30.085,   36.375,  202.68 ,   63.23 , 1332.65 ,  140.335,
          58.82 ,  200.515,   19.71 ,  112.33 ,  210.23 ,  135.415,
         117.915,   79.68 ,   73.905,  230.02 ,   27.045,  148.905,
          45.47 ,   10.29 ,  183.25 ,  387.765,  269.125,  184.39 ,
         176.515,   85.27 ,  286.32 ,   84.21 ,    6.735,  131.27 ,
          37.785,   87.1  ,  179.82 ,   34.92 ,  120.815,   63.29 ,
         527.2  ,   48.475,   55.845,  121.405,   30.685,  121.725,
         757.875,   52.44 ,  156.095,   93.445,  175.26 ,  121.08 ]])

In [101]:
X.loc[indices.sort_values(['days','timestemps','ticker']).index,:].head(66)['close_lag0'].values==data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1'])[['close']][(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(1)==1)&(data.reset_index().sort_values(['level_0','level_1','Ticker']).set_index(['level_0','level_1']).index.get_level_values(0)==20200501)].values.T

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True]])

In [104]:
indices.sort_values(['days','timestemps','ticker']).index

Int64Index([     0,   7798,  15596,  23394,  31192,  38990,  46788,  54586,
             62384,  70182,
            ...
            444485, 452283, 460081, 467879, 475677, 483475, 491273, 499071,
            506869, 514667],
           dtype='int64', length=514668)

In [105]:
indices.loc[7798,:]

days          20200501
timestemps           0
ticker            ABBV
Name: 7798, dtype: object

In [111]:
X.loc[7798,:][['close_lag0']]

close_lag0    81.76
Name: 7798, dtype: object

In [110]:
X_temp.loc[7798,:][['close_lag0']]

close_lag0    81.76
Name: 7798, dtype: object

In [112]:
X.loc[indices.sort_values(['days','timestemps','ticker']).index,:].head(66)

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
7798,-0.750,0.310,-0.750,81.760,0.24,1.06,0.07,0.80,14.0,20.0,...,0,0,0,0,1,0,0,0,0,ABBV
15596,-0.030,0.160,-0.320,91.220,0.18,0.58,0.05,0.12,1.0,3.0,...,0,0,0,0,1,0,0,0,0,ABT
23394,0.105,0.105,-0.100,82.370,0.13,0.54,0.10,0.26,1.0,2.0,...,0,0,0,0,0,0,0,0,1,AEP
31192,-0.335,0.685,-0.380,233.540,2.39,3.99,1.42,2.10,1.0,1.0,...,0,0,0,0,0,0,1,0,0,AMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475678,-0.110,0.070,-0.115,52.440,0.06,0.17,0.01,0.08,1.0,14.0,...,0,0,0,0,0,0,0,1,0,TSM
483476,0.670,0.720,-0.110,156.095,0.47,0.71,0.22,0.25,1.0,2.0,...,0,0,0,0,0,1,0,0,0,UNP
491274,-0.430,0.000,-0.710,93.445,1.03,1.10,0.05,0.65,1.0,6.0,...,0,0,0,0,0,1,0,0,0,UPS
499072,-0.365,0.170,-0.365,175.260,0.15,0.23,0.01,0.06,1.0,4.0,...,0,0,0,1,0,0,0,0,0,V


In [102]:
y.loc[indices.sort_values(['days','timestemps','ticker']).index,:].head(66)

,0
0,0.0
7798,0.0
15596,0.0
23394,0.0
31192,1.0
...,...
475678,1.0
483476,1.0
491274,0.0
499072,1.0


In [103]:
y

,0
0,0.0
1,1.0
2,1.0
3,0.0
4,1.0
...,...
514663,1.0
514664,0.0
514665,1.0
514666,0.0


In [11]:
int(data.shape[0]*0.8)

343200.0

In [13]:
data.reset_index().sort_values(['level_0',
                                'level_1',
                                'Ticker']).set_index(['level_0',
                                                      'level_1']).iloc[int(data.shape[0]*0.8):,:]

open     high      low    close  spread_open  \
level_0  level_1                                                    
20200526 0        323.560  324.195  323.380  323.875         0.46   
         0         93.625   93.625   93.060   93.295         0.35   
         0         92.915   93.255   92.815   93.025         0.77   
         0         79.450   80.200   78.745   79.645         0.66   
         0        248.750  248.750  246.555  248.300         2.10   
...                   ...      ...      ...      ...          ...   
20200529 389       50.455   50.495   50.325   50.325         0.01   
         389      169.960  170.030  169.610  169.675         0.04   
         389      100.095  100.110   99.670   99.670         0.05   
         389      195.855  195.920  194.195  194.195         0.03   
         389      124.075  124.225  122.810  123.855         0.01   

                  spread_high  spread_low  spread_close  bidsize_open  \
level_0  level_1                                                        
20200526 0               0.46        0.01          0.11           8.0   
         0               0.38        0.01          0.03           1.0   
         0               0.77        0.01          0.19           3.0   
         0               2.07        0.21          0.21           1.0   
         0               3.45        0.04          0.90           2.0   
...                       ...         ...           ...           ...   
20200529 389             0.05        0.01          0.05           5.0   
         389             0.36        0.01          0.23          10.0   
         389             0.13        0.01          0.12           1.0   
         389             2.39        0.01          2.39           2.0   
         389             2.43        0.01          0.21           1.0   

                  bidsize_high  bidsize_low  bidsize_close  ofrsize_open  \
level_0  level_1                                                           
20200526 0                18.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           3.0   
         0                 5.0          1.0            1.0           2.0   
         0                 5.0          1.0            1.0           1.0   
...                        ...          ...            ...           ...   
20200529 389              38.0          1.0            2.0           7.0   
         389              11.0          1.0            1.0           1.0   
         389              11.0          1.0            7.0           1.0   
         389               7.0          1.0            1.0           1.0   
         389              20.0          1.0            2.0           4.0   

                  ofrsize_high  ofrsize_low  ofrsize_close Ticker  
level_0  level_1                                                   
20200526 0               241.0          1.0            1.0   AAPL  
         0                10.0          1.0            1.0   ABBV  
         0                12.0          1.0            2.0    ABT  
         0                 5.0          1.0            2.0    AEP  
         0                 2.0          1.0            2.0    AMT  
...                        ...          ...            ...    ...  
20200529 389              24.0          1.0            9.0    TSM  
         389              10.0          1.0            2.0    UNP  
         389              11.0          1.0            1.0    UPS  
         389               9.0          1.0            2.0      V  
         389              12.0          1.0            1.0    WMT  

[85800 rows x 17 columns]

In [35]:
data.index.get_level_values(0).unique()

Int64Index([20200501, 20200504, 20200505, 20200506, 20200507, 20200508,
            20200511, 20200512, 20200513, 20200514, 20200515, 20200518,
            20200519, 20200520, 20200521, 20200522, 20200526, 20200527,
            20200528, 20200529],
           dtype='int64')

In [36]:
data = data.reset_index().sort_values(['level_0',
                                'level_1',
                                'Ticker']).set_index(['level_0',
                                                      'level_1'])
first_val_day = int(np.floor(data.index.get_level_values(0).unique().shape[0]*0.8))

X_train = data.loc[(data.index.get_level_values(0).unique()[:first_val_day],)]
X_test = data.loc[(data.index.get_level_values(0).unique()[first_val_day:],)]

# first_val_day = int(np.floor(data.reset_index().loc[:,'level_0'].unique().shape[0]*0.8))

# data.reset_index().loc[:,'level_0'].unique()[0:first_val_day]
# # data.reset_index().loc[:,'level_0'].unique().shape

In [42]:
X_train[X_train.Ticker=='AAPL']

open     high      low    close  spread_open  \
level_0  level_1                                                    
20200501 0        286.250  289.260  285.870  289.260         0.50   
         1        289.260  289.350  288.365  289.020         0.24   
         2        289.035  289.705  288.280  288.580         0.07   
         3        288.485  289.315  288.280  289.095         0.49   
         4        289.100  290.435  288.940  290.320         0.16   
...                   ...      ...      ...      ...          ...   
20200522 385      318.335  318.455  318.155  318.340         0.03   
         386      318.330  318.545  318.265  318.475         0.04   
         387      318.470  318.695  318.410  318.690         0.04   
         388      318.690  318.935  318.635  318.850         0.02   
         389      318.850  319.200  318.620  318.850         0.04   

                  spread_high  spread_low  spread_close  bidsize_open  \
level_0  level_1                                                        
20200501 0               0.50        0.01          0.24           6.0   
         1               0.45        0.01          0.10           9.0   
         2               0.49        0.01          0.30           1.0   
         3               0.49        0.01          0.17           1.0   
         4               0.33        0.01          0.10          13.0   
...                       ...         ...           ...           ...   
20200522 385             0.08        0.01          0.02           1.0   
         386             0.08        0.01          0.03           1.0   
         387             0.07        0.01          0.02           1.0   
         388             0.06        0.01          0.04           3.0   
         389             0.29        0.01          0.08           5.0   

                  bidsize_high  bidsize_low  bidsize_close  ofrsize_open  \
level_0  level_1                                                           
20200501 0                95.0          1.0           10.0           1.0   
         1                20.0          1.0            1.0           4.0   
         2                50.0          1.0            1.0           1.0   
         3                25.0          1.0           16.0           1.0   
         4                71.0          1.0            1.0           1.0   
...                        ...          ...            ...           ...   
20200522 385               9.0          1.0            1.0           2.0   
         386               9.0          1.0            2.0           3.0   
         387              12.0          1.0            3.0           2.0   
         388              12.0          1.0            5.0          19.0   
         389              55.0          1.0            4.0           2.0   

                  ofrsize_high  ofrsize_low  ofrsize_close Ticker  
level_0  level_1                                                   
20200501 0                85.0          1.0            4.0   AAPL  
         1                56.0          1.0            1.0   AAPL  
         2                13.0          1.0            1.0   AAPL  
         3                 8.0          1.0            1.0   AAPL  
         4               236.0          1.0            1.0   AAPL  
...                        ...          ...            ...    ...  
20200522 385              13.0          1.0            3.0   AAPL  
         386              44.0          1.0            2.0   AAPL  
         387              98.0          1.0           20.0   AAPL  
         388              34.0          1.0            3.0   AAPL  
         389             260.0          1.0           23.0   AAPL  

[6240 rows x 17 columns]

In [39]:
X_test

open     high      low    close  spread_open  \
level_0  level_1                                                    
20200526 0        323.560  324.195  323.380  323.875         0.46   
         0         93.625   93.625   93.060   93.295         0.35   
         0         92.915   93.255   92.815   93.025         0.77   
         0         79.450   80.200   78.745   79.645         0.66   
         0        248.750  248.750  246.555  248.300         2.10   
...                   ...      ...      ...      ...          ...   
20200529 389       50.455   50.495   50.325   50.325         0.01   
         389      169.960  170.030  169.610  169.675         0.04   
         389      100.095  100.110   99.670   99.670         0.05   
         389      195.855  195.920  194.195  194.195         0.03   
         389      124.075  124.225  122.810  123.855         0.01   

                  spread_high  spread_low  spread_close  bidsize_open  \
level_0  level_1                                                        
20200526 0               0.46        0.01          0.11           8.0   
         0               0.38        0.01          0.03           1.0   
         0               0.77        0.01          0.19           3.0   
         0               2.07        0.21          0.21           1.0   
         0               3.45        0.04          0.90           2.0   
...                       ...         ...           ...           ...   
20200529 389             0.05        0.01          0.05           5.0   
         389             0.36        0.01          0.23          10.0   
         389             0.13        0.01          0.12           1.0   
         389             2.39        0.01          2.39           2.0   
         389             2.43        0.01          0.21           1.0   

                  bidsize_high  bidsize_low  bidsize_close  ofrsize_open  \
level_0  level_1                                                           
20200526 0                18.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           3.0   
         0                 5.0          1.0            1.0           2.0   
         0                 5.0          1.0            1.0           1.0   
...                        ...          ...            ...           ...   
20200529 389              38.0          1.0            2.0           7.0   
         389              11.0          1.0            1.0           1.0   
         389              11.0          1.0            7.0           1.0   
         389               7.0          1.0            1.0           1.0   
         389              20.0          1.0            2.0           4.0   

                  ofrsize_high  ofrsize_low  ofrsize_close Ticker  
level_0  level_1                                                   
20200526 0               241.0          1.0            1.0   AAPL  
         0                10.0          1.0            1.0   ABBV  
         0                12.0          1.0            2.0    ABT  
         0                 5.0          1.0            2.0    AEP  
         0                 2.0          1.0            2.0    AMT  
...                        ...          ...            ...    ...  
20200529 389              24.0          1.0            9.0    TSM  
         389              10.0          1.0            2.0    UNP  
         389              11.0          1.0            1.0    UPS  
         389               9.0          1.0            2.0      V  
         389              12.0          1.0            1.0    WMT  

[85800 rows x 17 columns]

In [18]:
data.reset_index().loc[:,'level_0'].unique()[first_val_day:]

array([20200526, 20200527, 20200528, 20200529], dtype=int64)

In [21]:

data.reset_index().sort_values(['level_0',
                                'level_1',
                                'Ticker']).set_index(['level_0',
                                                      'level_1']).index.get_level_values(0)

Int64Index([20200501, 20200501, 20200501, 20200501, 20200501, 20200501,
            20200501, 20200501, 20200501, 20200501,
            ...
            20200529, 20200529, 20200529, 20200529, 20200529, 20200529,
            20200529, 20200529, 20200529, 20200529],
           dtype='int64', name='level_0', length=429000)

In [28]:
list(data.reset_index().loc[:,'level_0'].unique()[first_val_day:])

[20200526, 20200527, 20200528, 20200529]

In [29]:
data.reset_index().loc[:,'level_0'].unique()[first_val_day:]

AttributeError: 'numpy.ndarray' object has no attribute 'tuple'

In [30]:
data[(list(data.reset_index().loc[:,'level_0'].unique()[first_val_day:]),)]

TypeError: '([20200526, 20200527, 20200528, 20200529],)' is an invalid key

In [33]:
data.reset_index().loc[(data.reset_index().loc[:,'level_0'].unique()[first_val_day:],)]

open     high      low    close  spread_open  \
level_0  level_1                                                    
20200526 0        323.560  324.195  323.380  323.875         0.46   
         0         93.625   93.625   93.060   93.295         0.35   
         0         92.915   93.255   92.815   93.025         0.77   
         0         79.450   80.200   78.745   79.645         0.66   
         0        248.750  248.750  246.555  248.300         2.10   
...                   ...      ...      ...      ...          ...   
20200529 389       50.455   50.495   50.325   50.325         0.01   
         389      169.960  170.030  169.610  169.675         0.04   
         389      100.095  100.110   99.670   99.670         0.05   
         389      195.855  195.920  194.195  194.195         0.03   
         389      124.075  124.225  122.810  123.855         0.01   

                  spread_high  spread_low  spread_close  bidsize_open  \
level_0  level_1                                                        
20200526 0               0.46        0.01          0.11           8.0   
         0               0.38        0.01          0.03           1.0   
         0               0.77        0.01          0.19           3.0   
         0               2.07        0.21          0.21           1.0   
         0               3.45        0.04          0.90           2.0   
...                       ...         ...           ...           ...   
20200529 389             0.05        0.01          0.05           5.0   
         389             0.36        0.01          0.23          10.0   
         389             0.13        0.01          0.12           1.0   
         389             2.39        0.01          2.39           2.0   
         389             2.43        0.01          0.21           1.0   

                  bidsize_high  bidsize_low  bidsize_close  ofrsize_open  \
level_0  level_1                                                           
20200526 0                18.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           1.0   
         0                 8.0          1.0            1.0           3.0   
         0                 5.0          1.0            1.0           2.0   
         0                 5.0          1.0            1.0           1.0   
...                        ...          ...            ...           ...   
20200529 389              38.0          1.0            2.0           7.0   
         389              11.0          1.0            1.0           1.0   
         389              11.0          1.0            7.0           1.0   
         389               7.0          1.0            1.0           1.0   
         389              20.0          1.0            2.0           4.0   

                  ofrsize_high  ofrsize_low  ofrsize_close Ticker  
level_0  level_1                                                   
20200526 0               241.0          1.0            1.0   AAPL  
         0                10.0          1.0            1.0   ABBV  
         0                12.0          1.0            2.0    ABT  
         0                 5.0          1.0            2.0    AEP  
         0                 2.0          1.0            2.0    AMT  
...                        ...          ...            ...    ...  
20200529 389              24.0          1.0            9.0    TSM  
         389              10.0          1.0            2.0    UNP  
         389              11.0          1.0            1.0    UPS  
         389               9.0          1.0            2.0      V  
         389              12.0          1.0            1.0    WMT  

[85800 rows x 17 columns]

In [46]:
for i in X.ticker.unique():
    
    print(i,' - shape: ',X[X.ticker=='AAPL'].shape[0])

AAPL  - shape:  7798
ABBV  - shape:  7798
ABT  - shape:  7798
AEP  - shape:  7798
AMT  - shape:  7798
APD  - shape:  7798
BA  - shape:  7798
BABA  - shape:  7798
BAC  - shape:  7798
BHP  - shape:  7798
BP  - shape:  7798
CCI  - shape:  7798
CHL  - shape:  7798
COST  - shape:  7798
CSGP  - shape:  7798
D  - shape:  7798
DIS  - shape:  7798
ECL  - shape:  7798
ENB  - shape:  7798
EXC  - shape:  7798
FB  - shape:  7798
FMX  - shape:  7798
GOOG  - shape:  7798
INTC  - shape:  7798
JNJ  - shape:  7798
KO  - shape:  7798
LFC  - shape:  7798
LIN  - shape:  7798
LMT  - shape:  7798
MA  - shape:  7798
MCD  - shape:  7798
MSFT  - shape:  7798
NKE  - shape:  7798
NVDA  - shape:  7798
NVS  - shape:  7798
PBR  - shape:  7798
PEP  - shape:  7798
PFE  - shape:  7798
PLD  - shape:  7798
PSA  - shape:  7798
PTR  - shape:  7798
PYPL  - shape:  7798
RTX  - shape:  7798
SHW  - shape:  7798
SNP  - shape:  7798
SO  - shape:  7798
SRE  - shape:  7798
T  - shape:  7798
TM  - shape:  7798
TSLA  - shape:  7798


In [49]:
X.columns

Index(['open_lag0', 'high_lag0', 'low_lag0', 'close_lag0', 'spread_open_lag0',
       'spread_high_lag0', 'spread_low_lag0', 'spread_close_lag0',
       'bidsize_open_lag0', 'bidsize_high_lag0', 'bidsize_low_lag0',
       'bidsize_close_lag0', 'ofrsize_open_lag0', 'ofrsize_high_lag0',
       'ofrsize_low_lag0', 'ofrsize_close_lag0', 'open_lag1', 'high_lag1',
       'low_lag1', 'close_lag1', 'spread_open_lag1', 'spread_high_lag1',
       'spread_low_lag1', 'spread_close_lag1', 'bidsize_open_lag1',
       'bidsize_high_lag1', 'bidsize_low_lag1', 'bidsize_close_lag1',
       'ofrsize_open_lag1', 'ofrsize_high_lag1', 'ofrsize_low_lag1',
       'ofrsize_close_lag1', 'sector_Basic Materials',
       'sector_Communication Services', 'sector_Consumer Cyclical',
       'sector_Consumer Defensive', 'sector_Energy',
       'sector_Financial Services', 'sector_Healthcare', 'sector_Industrials',
       'sector_Real Estate', 'sector_Technology', 'sector_Utilities',
       'ticker'],
      dtype='obj

In [51]:
X.iloc[:,:-1].values

array([[ 0.24 ,  0.33 , -0.655, ...,  0.   ,  1.   ,  0.   ],
       [ 0.455,  1.125, -0.3  , ...,  0.   ,  1.   ,  0.   ],
       [-0.61 ,  0.22 , -0.815, ...,  0.   ,  1.   ,  0.   ],
       ...,
       [ 0.09 ,  0.09 , -0.075, ...,  0.   ,  0.   ,  0.   ],
       [-0.34 ,  0.02 , -0.35 , ...,  0.   ,  0.   ,  0.   ],
       [ 0.26 ,  0.28 , -0.015, ...,  0.   ,  0.   ,  0.   ]])

In [48]:
X.values.shape

(428890, 44)

In [52]:
test = pd.DataFrame({'container':X.iloc[:,:-1].values,'ticker':X.ticker})

Exception: Data must be 1-dimensional

# Testing a method based on no-resetting

In [58]:
X_temp.reset_index().sort_values(['index','ticker'])

,index,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,...,0,0,0,0,0,0,0,1,0,AAPL
7798,0,-0.750,0.310,-0.750,81.760,0.24,1.06,0.07,0.80,14.0,...,0,0,0,0,1,0,0,0,0,ABBV
15596,0,-0.030,0.160,-0.320,91.220,0.18,0.58,0.05,0.12,1.0,...,0,0,0,0,1,0,0,0,0,ABT
23394,0,0.105,0.105,-0.100,82.370,0.13,0.54,0.10,0.26,1.0,...,0,0,0,0,0,0,0,0,1,AEP
31192,0,-0.335,0.685,-0.380,233.540,2.39,3.99,1.42,2.10,1.0,...,0,0,0,0,0,0,1,0,0,AMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397697,7797,0.090,0.095,0.000,50.455,0.01,0.03,0.01,0.01,11.0,...,0,0,0,0,0,0,0,1,0,TSM
405495,7797,0.015,0.070,-0.070,169.965,0.06,0.12,0.01,0.03,2.0,...,0,0,0,0,0,1,0,0,0,UNP
413293,7797,0.075,0.095,-0.030,100.075,0.04,0.09,0.01,0.01,1.0,...,0,0,0,0,0,1,0,0,0,UPS
421091,7797,0.000,0.120,-0.055,195.850,0.08,0.12,0.01,0.04,1.0,...,0,0,0,1,0,0,0,0,0,V


In [60]:
X_temp.reset_index().loc[:,'index'].unique()

array([   0,    1,    2, ..., 7795, 7796, 7797], dtype=int64)

# Attemp to do it by day

In [61]:
data[['open','high','low','close','Ticker']]

open     high      low    close Ticker
level_0  level_1                                           
20200501 0        286.250  289.260  285.870  289.260   AAPL
         0         81.000   82.110   80.580   81.010   ABBV
         0         91.025   91.435   90.490   91.220    ABT
         0         83.455   83.980   82.250   82.470    AEP
         0        234.350  234.575  232.215  232.950    AMT
...                   ...      ...      ...      ...    ...
20200529 389       50.455   50.495   50.325   50.325    TSM
         389      169.960  170.030  169.610  169.675    UNP
         389      100.095  100.110   99.670   99.670    UPS
         389      195.855  195.920  194.195  194.195      V
         389      124.075  124.225  122.810  123.855    WMT

[429000 rows x 5 columns]

## Splitting the data

In [5]:
# Let's have a proper split (along tickers & dates)
train_size = 0.8
data_splits = pd.DataFrame()
data_splits = X.index.to_series().groupby(X['ticker']).agg(['first','last']).reset_index()

data_splits['val_size'] = ((1-train_size) * (data_splits['last'] - data_splits['first'])).astype(int)
data_splits['val_start_idx'] = data_splits['last'] - data_splits['val_size']
data_splits['val_end_idx'] = data_splits['last'] + 1 # to get the last observation included

data_splits['train_start_idx'] =  data_splits['first']
data_splits['train_end_idx'] = data_splits['val_start_idx']

# Store ranges

train_ranges = [list(x) for x in zip(data_splits['train_start_idx'], data_splits['train_end_idx'])]
val_ranges = [list(x) for x in zip(data_splits['val_start_idx'], data_splits['val_end_idx'])]


if verbose:
    data_splits

INFO:MainThread:numexpr.utils:NumExpr defaulting to 4 threads.


## Adding ticker dummies

In [6]:
## Adding ticker dummies
tickers = X.pop('ticker')
X = pd.concat([X, pd.get_dummies(tickers, prefix='ticker', drop_first=False)], axis=1)

In [13]:
X.columns

Index(['open_lag0', 'high_lag0', 'low_lag0', 'close_lag0', 'spread_open_lag0',
       'spread_high_lag0', 'spread_low_lag0', 'spread_close_lag0',
       'bidsize_open_lag0', 'bidsize_high_lag0', 'bidsize_low_lag0',
       'bidsize_close_lag0', 'ofrsize_open_lag0', 'ofrsize_high_lag0',
       'ofrsize_low_lag0', 'ofrsize_close_lag0', 'open_lag1', 'high_lag1',
       'low_lag1', 'close_lag1', 'spread_open_lag1', 'spread_high_lag1',
       'spread_low_lag1', 'spread_close_lag1', 'bidsize_open_lag1',
       'bidsize_high_lag1', 'bidsize_low_lag1', 'bidsize_close_lag1',
       'ofrsize_open_lag1', 'ofrsize_high_lag1', 'ofrsize_low_lag1',
       'ofrsize_close_lag1', 'sector_Basic Materials',
       'sector_Communication Services', 'sector_Consumer Cyclical',
       'sector_Consumer Defensive', 'sector_Energy',
       'sector_Financial Services', 'sector_Healthcare', 'sector_Industrials',
       'sector_Real Estate', 'sector_Technology', 'sector_Utilities',
       'ticker_AAPL', 'ticker_ABBV

In [21]:
train_ranges[0]

[0, 6238]

## Constructing our final train/validation sets

In [39]:
train_ds = pd.concat([X.iloc[start:end, :] for (start, end) in train_ranges]).reset_index(drop=True)
train_y = pd.concat([y.iloc[start:end] for (start, end) in train_ranges]).reset_index(drop=True)

validate_ds = pd.concat([X.iloc[start:end, :] for (start, end) in val_ranges]).reset_index(drop=True)
val_y = pd.concat([y.iloc[start:end] for (start, end) in val_ranges]).reset_index(drop=True)

train_ds.shape, train_y.shape, validate_ds.shape, val_y.shape, train_y.shape[0] + val_y.shape[0]

((343090, 98), (343090, 1), (85800, 98), (85800, 1), 428890)

In [40]:
train_ranges[0]

[0, 6238]

In [41]:
train_ranges[0][1]+10

6248

## Pre-processing

In [44]:
{i:colname for i,colname in enumerate(train_ds.columns)}

{0: 'open_lag0',
 1: 'high_lag0',
 2: 'low_lag0',
 3: 'close_lag0',
 4: 'spread_open_lag0',
 5: 'spread_high_lag0',
 6: 'spread_low_lag0',
 7: 'spread_close_lag0',
 8: 'bidsize_open_lag0',
 9: 'bidsize_high_lag0',
 10: 'bidsize_low_lag0',
 11: 'bidsize_close_lag0',
 12: 'ofrsize_open_lag0',
 13: 'ofrsize_high_lag0',
 14: 'ofrsize_low_lag0',
 15: 'ofrsize_close_lag0',
 16: 'open_lag1',
 17: 'high_lag1',
 18: 'low_lag1',
 19: 'close_lag1',
 20: 'spread_open_lag1',
 21: 'spread_high_lag1',
 22: 'spread_low_lag1',
 23: 'spread_close_lag1',
 24: 'bidsize_open_lag1',
 25: 'bidsize_high_lag1',
 26: 'bidsize_low_lag1',
 27: 'bidsize_close_lag1',
 28: 'ofrsize_open_lag1',
 29: 'ofrsize_high_lag1',
 30: 'ofrsize_low_lag1',
 31: 'ofrsize_close_lag1',
 32: 'sector_Basic Materials',
 33: 'sector_Communication Services',
 34: 'sector_Consumer Cyclical',
 35: 'sector_Consumer Defensive',
 36: 'sector_Energy',
 37: 'sector_Financial Services',
 38: 'sector_Healthcare',
 39: 'sector_Industrials',
 40: 

In [47]:
# Creating one ppdict for individual preprocessings
# ppdict1 = {'open':'minmax',
#           'high':'log',
#           'low':'log',
#           'close':'std'}
splitpoint = 32

# Standardize some features
ppdict1 = {i:'std' for i in train_ds.columns[0:splitpoint]} 
# Keep some in actual levels (Dummies in this case).
ppdict2 = {i:'act' for i in train_ds.columns[splitpoint:]} 

# Merging the two
ppdict = {**ppdict1,**ppdict2}

In [54]:
train_ds,validate_ds = pre_processing(train_ds,
                                    validate_ds,
                                    ppdict,
                                    100,
                                    verbose =True)

Pre-Processing Procedure:  act
Columns Processed: ['sector_Basic Materials' 'sector_Communication Services'
 'sector_Consumer Cyclical' 'sector_Consumer Defensive' 'sector_Energy'
 'sector_Financial Services' 'sector_Healthcare' 'sector_Industrials'
 'sector_Real Estate' 'sector_Technology' 'sector_Utilities' 'ticker_AAPL'
 'ticker_ABBV' 'ticker_ABT' 'ticker_AEP' 'ticker_AMT' 'ticker_APD'
 'ticker_BA' 'ticker_BABA' 'ticker_BAC' 'ticker_BHP' 'ticker_BP'
 'ticker_CCI' 'ticker_CHL' 'ticker_COST' 'ticker_CSGP' 'ticker_D'
 'ticker_DIS' 'ticker_ECL' 'ticker_ENB' 'ticker_EXC' 'ticker_FB'
 'ticker_FMX' 'ticker_GOOG' 'ticker_INTC' 'ticker_JNJ' 'ticker_KO'
 'ticker_LFC' 'ticker_LIN' 'ticker_LMT' 'ticker_MA' 'ticker_MCD'
 'ticker_MSFT' 'ticker_NKE' 'ticker_NVDA' 'ticker_NVS' 'ticker_PBR'
 'ticker_PEP' 'ticker_PFE' 'ticker_PLD' 'ticker_PSA' 'ticker_PTR'
 'ticker_PYPL' 'ticker_RTX' 'ticker_SHW' 'ticker_SNP' 'ticker_SO'
 'ticker_SRE' 'ticker_T' 'ticker_TM' 'ticker_TSLA' 'ticker_TSM'
 'ticker_UNP' 't

In [53]:
ppX_train.iloc[:,0].mean(),ppX_train.iloc[:,0].std()

(-1.8927265537610815e-16, 1.000001457346533)

## Prepping for models

In [8]:
N_VALIDATION = val_y.shape[0] #int(1e3)
N_TRAIN = train_y.shape[0] #int(1e4)
# BUFFER_SIZE = int(1e4)
BATCH_SIZE = 256 #512 #32
MAX_EPOCHS = 500

STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_REPEAT = int(N_TRAIN / ((STEPS_PER_EPOCH * MAX_EPOCHS) / BATCH_SIZE))
FEATURES = X.shape[1]

N_TRAIN, N_VALIDATION, N_TRAIN + N_VALIDATION, STEPS_PER_EPOCH, N_REPEAT, STEPS_PER_EPOCH * MAX_EPOCHS

(343090, 85800, 428890, 1340, 131, 670000)

## A Logistic Regression model in TF/Keras

In [55]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      #keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# def make_model(metrics = METRICS, output_bias=None):
#   if output_bias is not None:
#     output_bias = tf.keras.initializers.Constant(output_bias)
#   model = keras.Sequential([
#       keras.layers.Dense(
#           16, activation='relu',
#           input_shape=(train_features.shape[-1],)),
#       keras.layers.Dropout(0.5),
#       keras.layers.Dense(1, activation='sigmoid',
#                          bias_initializer=output_bias),
#   ])

#   model.compile(
#       optimizer=keras.optimizers.Adam(lr=1e-3),
#       loss=keras.losses.BinaryCrossentropy(),
#       metrics=metrics)

#   return model

# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,))
# })

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10)
    keras.layers.Dense(1,
                       input_shape=(FEATURES,),
                       activation='sigmoid',
                       kernel_regularizer=regularizers.l2(1))
])

model.summary()

# with final activation (Keras/TF tutorial advises against this practice, but they also use it later in the tutorial)
# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,), activation='sigmoid')
# })

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', ])
model.compile(
              optimizer=keras.optimizers.Adam(), #lr=1e-3
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)

early_stopping = tf.keras.callbacks.EarlyStopping(
                                                monitor='val_auc', 
                                                verbose=1,
                                                patience=100,
                                                mode='max',
                                                restore_best_weights=True)

def get_callbacks(run_id):
      return [
             tfdocs.modeling.EpochDots(),
             early_stopping,
             tf.keras.callbacks.TensorBoard(logdir), #/run_id),
      ]

baseline_history = model.fit(
                            train_ds, #train_features,
                            train_y, #train_labels,
                            batch_size=512, #BATCH_SIZE,
                            epochs=1000, #EPOCHS,
                            callbacks = get_callbacks(run_id = 'first'), #[early_stopping],
                            validation_data=(validate_ds, val_y),
                            verbose=0) #(val_features, val_labels))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 99        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________



Epoch: 0, accuracy:0.5352,  auc:0.5034,  loss:0.8996,  val_accuracy:0.5456,  val_auc:0.5453,  val_loss:0.6876,  
....................................................................................................
Epoch: 100, accuracy:0.5480,  auc:0.5440,  loss:0.6873,  val_accuracy:0.5454,  val_auc:0.5459,  val_loss:0.6879,  
..................Restoring model weights from the end of the best epoch.
Epoch 00118: early stopping


In [56]:
model.evaluate(validate_ds,  val_y, verbose=2)

2682/2682 - 6s - loss: 0.6879 - accuracy: 0.5457 - auc: 0.5513


[0.6878659725189209, 0.5456876754760742, 0.5513222217559814]

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9296.